Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame.
Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [1]:
!pip install pyspark >> None

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg,col
import pyspark.sql.functions as F
from pyspark.sql import Window

In [3]:
# Создание сессии Spark
spark = SparkSession.builder.appName("window_average_revenue").getOrCreate()


In [4]:
# Загрузка данных о продажах
sales_data = [("2023-11-20", "Electronics", 100, 12000),
              ("2023-11-21", "Electronics", 110, 13000),
              ("2023-11-22", "Electronics", 105, 12500),
              ("2023-11-20", "Clothing", 300, 15000),
              ("2023-11-21", "Clothing", 280, 14000),
              ("2023-11-22", "Clothing", 320, 16000),
              ("2023-11-20", "Books", 150, 9000),
              ("2023-11-21", "Books", 200, 12000),
              ("2023-11-22", "Books", 180, 10000)
]

sales_df = spark.createDataFrame(sales_data, ["date", "category", "quantity", "revenue"])

In [6]:
# Оконная функция для нахождения среднего значения продаж для каждой категории продукта за каждый день
avg_sales_window = Window.partitionBy("category", "date").orderBy("date")

In [7]:
# Создаем новый столбец в датафрейме для нахождения среднего значения и используем оконную функцию
sales_df = sales_df.withColumn("avg_sales", F.avg("revenue").over(avg_sales_window))
sales_df.show()

+----------+-----------+--------+-------+---------+
|      date|   category|quantity|revenue|avg_sales|
+----------+-----------+--------+-------+---------+
|2023-11-20|      Books|     150|   9000|   9000.0|
|2023-11-21|      Books|     200|  12000|  12000.0|
|2023-11-22|      Books|     180|  10000|  10000.0|
|2023-11-20|   Clothing|     300|  15000|  15000.0|
|2023-11-21|   Clothing|     280|  14000|  14000.0|
|2023-11-22|   Clothing|     320|  16000|  16000.0|
|2023-11-20|Electronics|     100|  12000|  12000.0|
|2023-11-21|Electronics|     110|  13000|  13000.0|
|2023-11-22|Electronics|     105|  12500|  12500.0|
+----------+-----------+--------+-------+---------+



In [8]:
# Применение функции pivot
pivot_df = sales_df.groupBy("category").pivot("date").agg(F.avg("avg_sales"))
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+

